In [147]:
import numpy as np
import pandas as pd
import math

# s=pd.Series({'Google':np.random.normal(0, .2, 10),
#                            'Apple':np.random.normal(0, .2, 10),
#                            'Ebay':np.random.normal(0, .2, 10)})
s = pd.Series({'Google':[.2,.3,.4,-.2], 'Apple': [.3,.1,.3,.4],'Ebay':[.1,.7,-.1,-.2]})
x=s.values

# x = np.array([[0, 2 , 3], [1, 1, 6], [2, 0, 9]]).T

# np.cov(x)
# s = pd.Series({'a':[1,2], 'b':[2,3], 'c':[3,5]})
# u=np.array(s.values)
# u.astype(int)
# print(list(s.values))
returns=np.array(list(s.values))
cov_matrix=np.cov(np.array(list(s.values)))
print(cov)

[[ 0.02   0.01  -0.01 ]
 [ 0.01   0.005 -0.005]
 [-0.01  -0.005  0.005]]


In [148]:
# ticker= pd.Series({'Google':[1,2,3], 'Apple': [3,6,9], 'AMZN':[8,10,12]})
# cov_matrix=np.cov(np.array(list(ticker.values)))

In [149]:
diag=cov_matrix.diagonal()
diag_matrix=np.diag(np.diag(cov_matrix)) # gives n*n diagonal matrix
diag_matrix

array([[ 0.01583333,  0.        ,  0.        ],
       [ 0.        ,  0.1625    ,  0.        ],
       [ 0.        ,  0.        ,  0.06916667]])

In [150]:
from arch import arch_model
garch11 = arch_model(diag, p=1, q=1)
res = garch11.fit(update_freq=10)
print(res.summary())

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -4.19895560621
            Iterations: 6
            Function evaluations: 41
            Gradient evaluations: 6
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                      y   R-squared:                      -0.001
Mean Model:             Constant Mean   Adj. R-squared:                 -0.001
Vol Model:                      GARCH   Log-Likelihood:                4.19896
Distribution:                  Normal   AIC:                         -0.397911
Method:            Maximum Likelihood   BIC:                          -4.00346
                                        No. Observations:                    3
Date:                Thu, Feb 09 2017   Df Residuals:                       -1
Time:                        19:41:50   Df Model:                            4
                                  Mean Model                                  
     

In [151]:
alpha=res.params[2]
beta=res.params[3]
# std_err=np.random.normal(0, 1, 3) #epselon is the returns 

In [152]:
# esubt=np.linalg.inv(diag_matrix)
# ret=[.1,.2,.13] # just to see how it works
esubt=np.matmul(np.linalg.inv(diag_matrix),returns)
# esubt=np.matmul(np.linalg.inv(diag_matrix),returns)
esubt

array([[ 18.94736842,   6.31578947,  18.94736842,  25.26315789],
       [  0.61538462,   4.30769231,  -0.61538462,  -1.23076923],
       [  2.89156627,   4.3373494 ,   5.78313253,  -2.89156627]])

In [153]:
#not sure
qdash=np.cov(esubt)
q_start=np.ones((3,3))
qdash
# q_start

array([[ 63.15789474, -19.10931174, -19.02346227],
       [-19.10931174,   6.15384615,   4.22613531],
       [-19.02346227,   4.22613531,  14.45783133]])

In [154]:
q=((1-alpha-beta)*qdash) + (alpha*np.matmul(esubt,esubt.T)) + ((beta)*q_start)
q

array([[ 9.10497541, -1.62212019, -1.61092597],
       [-1.62212019,  1.67202718,  1.42066654],
       [-1.61092597,  1.42066654,  2.7548115 ]])

In [155]:
# r=np.matmul(np.linalg.inv(qdash),q)
qstar=np.diag(np.diag(cov_matrix))
qstar=np.sqrt(qstar)
qstar

array([[ 0.12583057,  0.        ,  0.        ],
       [ 0.        ,  0.40311289,  0.        ],
       [ 0.        ,  0.        ,  0.26299556]])

In [156]:
r=np.matmul(np.linalg.inv(qstar),q,np.linalg.inv(qstar))
r

array([[ 72.35900721, -12.89130408, -12.8023414 ],
       [ -4.02398495,   4.14778896,   3.52423995],
       [ -6.1252971 ,   5.40186504,  10.47474513]])

In [157]:
diag_matrix=np.diag(np.diag(cov_matrix))
print(diag_matrix)
predi_cov=np.matmul(diag_matrix,r)
predi_cov=np.matmul(predi_cov,diag_matrix)
print(predi_cov)

[[ 0.01583333  0.          0.        ]
 [ 0.          0.1625      0.        ]
 [ 0.          0.          0.06916667]]
[[ 0.01814    -0.03316825 -0.01402034]
 [-0.01035338  0.10952755  0.03961099]
 [-0.00670805  0.06071471  0.05011147]]


In [252]:
stock_returns = pd.Series({'Google':np.random.normal(0, .2, 10),
                           'Apple':np.random.normal(0, .2, 10),
                           'Ebay':np.random.normal(0, .2, 10)})


returns=np.array(list(stock_returns.values))
returns=returns[:,0:3]

#inititial condition setting esubt and q
esubt=[]
q_list=[]

cov_matrix=np.cov(returns)
diag=cov_matrix.diagonal()
diag_matrix=np.diag(np.diag(cov_matrix))
garch11 = arch_model(diag, p=1, q=1)
res = garch11.fit(update_freq=10)
alpha=res.params[2]
beta=res.params[3]

esubt.append(np.matmul(np.linalg.inv(diag_matrix),returns))
q_list.append(np.ones((3,3)))
pred_cov=[]

print(esubt)

def dccGARCH(returns):    
    cov_matrix=np.cov(returns)
    diag=cov_matrix.diagonal()
    diag_matrix=np.diag(np.diag(cov_matrix))
    
    garch11 = arch_model(diag, p=1, q=1)
    res = garch11.fit(update_freq=10)
    alpha=res.params[2]
    beta=res.params[3]    
    
    esubt.append(np.matmul(np.linalg.inv(diag_matrix),returns))
#     print(esubt)
#     print('\n')
#     print(np.matmul(esubt[-2],esubt[-2].T))
    qdash=np.cov(esubt[-2])
#    q_start=np.ones((3,3))
    
    q_list.append(((1-alpha-beta)*qdash) + 
             (alpha*np.matmul(esubt[-2],esubt[-2].T)) + ((beta)*q_list[-1]))
    
#     print("&&&&&")
    
    qstar=np.diag(np.diag(cov_matrix))
    qstar=np.sqrt(qstar)
    
    r=np.matmul(np.linalg.inv(qstar),q_list[-1],np.linalg.inv(qstar))
    
    pred_cov_temp=np.matmul(diag_matrix,r)
    pred_cov.append(np.matmul(pred_cov_temp,diag_matrix))
    
    print(pred_cov[-1])
    print(i)
    print('\n')
    

# dccGARCH(returns[:,0:4])
for i in range(4,10):
    dccGARCH(returns[:,0:i])

Iteration:     10,   Func. Count:     73,   Neg. LLF: -7.671355243057597
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -7.67136033697
            Iterations: 15
            Function evaluations: 88
            Gradient evaluations: 11
[array([[ 4.92369417, -2.29398584, -0.81756139],
       [-2.05152054,  8.80097191, -2.93300965],
       [ 8.42631211, -1.67835338,  7.74382541]])]
Iteration:     10,   Func. Count:     73,   Neg. LLF: -7.671355243057597
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -7.67136033697
            Iterations: 15
            Function evaluations: 88
            Gradient evaluations: 11
[[ 0.08660642 -0.02161622  0.04084096]
 [-0.03704704  0.04554062 -0.04586034]
 [ 0.06048639 -0.03963011  0.05371141]]
4


Iteration:     10,   Func. Count:     73,   Neg. LLF: -7.671355243057597
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -7.6713

In [253]:
returns=np.array(list(stock_returns.values))
returns=returns[:,0:9]
np.cov(returns)

array([[ 0.04037947, -0.01532836,  0.01774043],
       [-0.01532836,  0.02028582, -0.00258662],
       [ 0.01774043, -0.00258662,  0.03162873]])

In [254]:
def simulate_GARCH(T, a0, a1, b1, sigma1):
    
    # Initialize our values
    X = np.ndarray(T)
    sigma = np.ndarray(T)
    sigma[0] = sigma1
    
    for t in range(1, T):
        # Draw the next x_t
        X[t - 1] = sigma[t - 1] * np.random.normal(0, 1)
        # Draw the next sigma_t
        sigma[t] = math.sqrt(a0 + b1 * sigma[t - 1]**2 + a1 * X[t - 1]**2)
        
    X[T - 1] = sigma[T - 1] * np.random.normal(0, 1)    
    
    return X, sigma


In [256]:
returns=np.array(list(stock_returns.values))
# returns[1]
i=0
for n in returns:
#     print(n)
    print('\n')
# print(returns[1])
    garch11 = arch_model(n, p=1, q=1)
    res = garch11.fit(update_freq=10)
    omega=res.params[1]
    alpha=res.params[2]
    beta=res.params[3]
    # sig_ma=np.sqrt(np.mean(returns[1]))
    sig_ma=np.std(n)  # Assumption that I am making
    # print(sigma)
    # sig_ma=0
    # sigma = math.sqrt(omega / (1 - alpha - beta))
    print(n)
    print('\n')
    return_forecast,sigma_forecast=simulate_GARCH(5,omega,alpha,beta,sig_ma)
    n=np.append(n,return_forecast)
    # print(res.summary())
    print(n)



Optimization terminated successfully.    (Exit mode 0)
            Current function value: -3.07394744926
            Iterations: 9
            Function evaluations: 60
            Gradient evaluations: 9
[  3.38643579e-01  -1.57776569e-01  -5.62305261e-02  -5.08032898e-03
   1.66075324e-04   3.14044538e-01   2.94738921e-01   3.07734944e-01
  -6.69174505e-02   8.92326552e-02]


[  3.38643579e-01  -1.57776569e-01  -5.62305261e-02  -5.08032898e-03
   1.66075324e-04   3.14044538e-01   2.94738921e-01   3.07734944e-01
  -6.69174505e-02   8.92326552e-02  -5.17208194e-03   2.53082341e-01
  -6.71535463e-02   4.69033755e-02  -2.21097007e-02]


Iteration:     10,   Func. Count:     68,   Neg. LLF: -6.3091297331659675
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -6.30913288858
            Iterations: 11
            Function evaluations: 74
            Gradient evaluations: 11
[-0.04803749  0.20607963 -0.06867804 -0.00248202 -0.14289903 -0.28806574
 

In [240]:
import math
omega=res.params[1]
alpha=res.params[2]
beta=res.params[3]
# sig_ma=np.sqrt(np.mean(returns[1]))
sig_ma=np.std(returns[1])  # Assumption that I am making
# print(sigma)
# sig_ma=0
# sigma = math.sqrt(omega / (1 - alpha - beta))
return_forecast,sigma_forecast=simulate_GARCH(5,omega,alpha,beta,sig_ma)
return_forecast


array([ 0.04201855,  0.06044578, -0.68989632, -0.16610597, -0.00166613])

In [249]:
returns[1].size

10